# 🎭 Identity-Aware Segmentation with SAM 3 & InsightFace

**โปรเจค:** ระบบ Segmentation ที่รู้จำตัวตนสำหรับสมาชิกวง IVE

**เทคโนโลยี:**
- SAM 3 (Segment Anything Model 3) - สำหรับ Segmentation
- InsightFace - สำหรับ Face Recognition
- RTX 6000 (48GB VRAM) Optimized

**สมาชิก IVE 6 คน:** Wonyoung, Yujin, Gaeul, Liz, Leeseo, Rei

---
## 📋 Table of Contents

1. [Environment Setup](#section-1)
2. [Face Embedding Database](#section-2)
3. [Identity Matching](#section-3)
4. [SAM 3 Engine](#section-4)
5. [Integration Pipeline](#section-5)
6. [Gradio UI](#section-6)
7. [Video Inference](#section-7)

---
<a id='section-1'></a>
## Section 1: Environment Setup 🔧

ติดตั้ง dependencies และตั้งค่าสภาพแวดล้อมสำหรับ RTX 6000

In [ ]:
# =============================================================================
# CELL 1.1: Install Dependencies
# =============================================================================

# ติดตั้ง PyTorch 2.7.0 ที่รองรับ CUDA 12.1
!pip install -q torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# ติดตั้ง Face Recognition และ Segmentation libraries
!pip install -q insightface onnxruntime-gpu opencv-python Pillow

# ติดตั้ง UI และ Utilities
!pip install -q gradio matplotlib scikit-learn scipy
!pip install -q huggingface-hub transformers accelerate

# ติดตั้ง Video Processing
!pip install -q tqdm imageio imageio-ffmpeg av

print("✅ Dependencies ติดตั้งเสร็จสิ้น!")

In [ ]:
# =============================================================================
# CELL 1.2: Clone และติดตั้ง SAM 3
# =============================================================================

import os

# ตรวจสอบว่ามีโฟลเดอร์ sam3 อยู่หรือไม่
if not os.path.exists("sam3"):
    print("📥 กำลัง clone SAM 3 repository...")
    !git clone https://github.com/facebookresearch/sam3.git
    print("✅ Clone SAM 3 เสร็จสิ้น!")
else:
    print("📁 SAM 3 directory มีอยู่แล้ว")

# เข้าไปใน directory และติดตั้ง
%cd sam3
!pip install -q -e ".[notebooks]"
%cd ..

print("✅ ติดตั้ง SAM 3 เสร็จสิ้น!")

In [ ]:
# =============================================================================
# CELL 1.3: HuggingFace Login
# =============================================================================

from huggingface_hub import login

print("🔑 กรุณาใส่ HuggingFace Token ของคุณ")
print("   - ไปที่ https://huggingface.co/settings/tokens")
print("   - สร้าง token ใหม่ (ต้องมี access ถึง SAM 3)")
print("   - วาง token ที่นี่:")
login()

print("✅ Login HuggingFace สำเร็จ!")

In [ ]:
# =============================================================================
# CELL 1.4: Verify GPU และ CUDA
# =============================================================================

import torch

print("=" * 60)
print("🖥️  GPU Information")
print("=" * 60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU: {gpu_name}")
    
    cuda_version = torch.version.cuda
    print(f"📌 CUDA Version: {cuda_version}")
    
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"📌 Total VRAM: {total_memory:.2f} GB")
    
    major = torch.cuda.get_device_capability(0)[0]
    minor = torch.cuda.get_device_capability(0)[1]
    print(f"📌 Compute Capability: {major}.{minor}")
    
    if major >= 8:
        print("✅ bfloat16 supported!")
else:
    print("❌ CUDA not available!")

In [ ]:
# =============================================================================
# CELL 1.5: Import All Libraries
# =============================================================================

import torch
import torch.nn as nn
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

# Face Recognition
from insightface.app import FaceAnalysis

# SAM 3
from sam3 import SAM3ImagePredictor, build_sam3

# UI
import gradio as gr

# Utilities
from scipy.optimize import linear_sum_assignment
from tqdm import tqdm
import os

print("✅ Libraries imported successfully!")

---
<a id='section-2'></a>
## Section 2: Face Embedding Database 👤

สร้างฐานข้อมูล Face Embeddings สำหรับสมาชิก IVE 6 คน

In [ ]:
# =============================================================================
# SECTION 2: Face Embedding Database Creation
# =============================================================================

# กำหนด Mapping ชื่อสมาชิก
MEMBER_MAPPING = {
    "An_Yujin": "Yujin",
    "Jang_Wonyoung": "Wonyoung",
    "Kim_Gaeul": "Gaeul",
    "Kim_Jiwon": "Liz",
    "Lee_Hyunseo": "Leeseo",
    "Naoi_Rei": "Rei"
}

# โหลด InsightFace
print("🚀 กำลังโหลด InsightFace model (buffalo_l)...")
face_analyzer = FaceAnalysis(
    name='buffalo_l',
    root='./insightface_models',
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)
face_analyzer.prepare(ctx_id=0, det_size=(640, 640))
print("✅ InsightFace model พร้อมใช้งาน!")

def normalize_embedding(embedding):
    """ทำ L2 Normalization บน face embedding"""
    norm = np.linalg.norm(embedding)
    if norm < 1e-10:
        return embedding
    return embedding / norm

def create_embedding_database(dataset_path, face_analyzer, member_mapping):
    """สร้างฐานข้อมูล face embeddings"""
    embeddings_db = {}
    
    for folder_name, member_name in member_mapping.items():
        member_path = Path(dataset_path) / folder_name
        
        if not member_path.exists():
            print(f"❌ ไม่พบโฟลเดอร์: {member_path}")
            continue
        
        image_files = list(member_path.glob('*.jpg')) + list(member_path.glob('*.png'))
        member_embeddings = []
        
        for img_path in image_files:
            img_bgr = cv2.imread(str(img_path))
            if img_bgr is None:
                continue
            
            faces = face_analyzer.get(img_bgr)
            if len(faces) > 0:
                face = max(faces, key=lambda f: (f.bbox[2]-f.bbox[0])*(f.bbox[3]-f.bbox[1]))
                normalized_embedding = normalize_embedding(face.embedding)
                member_embeddings.append(normalized_embedding)
        
        if len(member_embeddings) > 0:
            avg_embedding = np.mean(member_embeddings, axis=0)
            avg_embedding = normalize_embedding(avg_embedding)
            embeddings_db[member_name] = avg_embedding
            print(f"✅ {member_name}: {len(member_embeddings)} faces extracted")
    
    return embeddings_db

# สร้าง embeddings database
embeddings_db = create_embedding_database("Dataset", face_analyzer, MEMBER_MAPPING)
print(f"\n✅ Face Embedding Database พร้อมใช้งาน! ({len(embeddings_db)} สมาชิก)")

---
<a id='section-3'></a>
## Section 3: Identity Matching 🎯

ฟังก์ชันสำหรับระบุตัวตนสมาชิก IVE จากใบหน้าในภาพ
ใช้ Cosine Similarity และ Hungarian Matching

In [ ]:
# =============================================================================
# SECTION 3: Identity Matching Functions
# =============================================================================

def cosine_similarity(embedding1, embedding2):
    """คำนวณ cosine similarity ระหว่าง 2 embeddings"""
    return np.dot(embedding1, embedding2)

def hungarian_matching(faces, embeddings_db, threshold=0.45):
    """
    ใช้ Hungarian Algorithm เพื่อหาการจับคู่ใบหน้ากับสมาชิกที่ optimal
    ป้องกันกรณีที่หลายใบหน้าถูกระบุเป็นคนเดียวกัน
    """
    if len(faces) == 0 or len(embeddings_db) == 0:
        return []
    
    member_names = list(embeddings_db.keys())
    member_embeddings = [embeddings_db[name] for name in member_names]
    
    num_faces = len(faces)
    num_members = len(member_names)
    
    # สร้าง Similarity Matrix
    similarity_matrix = np.zeros((num_faces, num_members))
    
    for i, face in enumerate(faces):
        face_embedding = face.embedding / (np.linalg.norm(face.embedding) + 1e-10)
        for j, member_emb in enumerate(member_embeddings):
            similarity = cosine_similarity(face_embedding, member_emb)
            similarity_matrix[i, j] = similarity
    
    # ใช้ Hungarian Algorithm
    cost_matrix = -similarity_matrix
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    matches = [None] * num_faces
    
    for face_idx, member_idx in zip(row_ind, col_ind):
        similarity = similarity_matrix[face_idx, member_idx]
        if similarity >= threshold:
            member_name = member_names[member_idx]
            matches[face_idx] = (face_idx, member_name, similarity)
    
    return matches

def identify_all_members(image_bgr, face_analyzer, embeddings_db, threshold=0.45):
    """ระบุตัวตนสมาชิก IVE ทั้งหมดในภาพ"""
    results = []
    
    faces = face_analyzer.get(image_bgr)
    
    if len(faces) == 0:
        print("ℹ️ ไม่พบใบหน้าในภาพ")
        return results
    
    print(f"🔍 พบ {len(faces)} ใบหน้าในภาพ")
    
    matches = hungarian_matching(faces, embeddings_db, threshold)
    
    for i, face in enumerate(faces):
        bbox = face.bbox.astype(int).tolist()
        
        if matches[i] is not None:
            _, member_name, similarity = matches[i]
            results.append({
                'name': member_name,
                'bbox': bbox,
                'similarity': float(similarity)
            })
            print(f"   ✅ {member_name}: similarity={similarity:.4f}")
        else:
            results.append({
                'name': 'Unknown',
                'bbox': bbox,
                'similarity': None
            })
    
    return results

print("✅ Identity matching functions loaded!")

---
<a id='section-4'></a>
## Section 4: SAM 3 Engine ✂️

โหลด SAM 3 และสร้างฟังก์ชันสำหรับ Segmentation

In [ ]:
# =============================================================================
# SECTION 4: SAM 3 Engine
# =============================================================================

# ตั้งค่า precision สูงสุดสำหรับ RTX 6000
torch.set_float32_matmul_precision('high')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 SAM 3 Engine: Using device = {DEVICE}")

# โหลด SAM 3 Model
print("\n📦 Loading SAM 3 Model...")

# ใช้ SAM 3 Large สำหรับความแม่นยำสูงสุด
sam3_checkpoint = "facebook/sam3-hiera-large"

from transformers import Sam3Model, Sam3Processor

sam3_processor = Sam3Processor.from_pretrained(sam3_checkpoint)
sam3_model = Sam3Model.from_pretrained(sam3_checkpoint).to(DEVICE)

# Optimize ด้วย bfloat16
sam3_model = sam3_model.to(torch.bfloat16)

# ใช้ torch.compile สำหรับ RTX 6000
sam3_model = torch.compile(sam3_model, mode="reduce-overhead")
sam3_model.eval()

print("✅ SAM 3 Model loaded successfully!")

def segment_by_box(image_pil, box_xyxy):
    """สร้าง mask จาก bounding box"""
    inputs = sam3_processor(
        images=image_pil,
        input_boxes=[[[box_xyxy]]],
        return_tensors="pt"
    ).to(DEVICE)
    
    with torch.no_grad():
        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = sam3_model(**inputs)
    
    masks = sam3_processor.image_processor.post_process_masks(
        outputs.pred_masks,
        inputs["original_sizes"],
        inputs["reshaped_input_sizes"]
    )
    
    mask_tensor = masks[0]
    iou_scores = outputs.iou_scores[0, 0]
    best_idx = torch.argmax(iou_scores).item()
    best_mask = mask_tensor[0, best_idx].cpu().numpy()
    
    return (best_mask > 0.5).astype(np.uint8)

print("✅ SAM 3 segmentation function ready!")

---
<a id='section-5'></a>
## Section 5: Integration Pipeline 🔗

เชื่อมโยง InsightFace กับ SAM 3 - The Magic Function

In [ ]:
# =============================================================================
# SECTION 5: Integration Pipeline
# =============================================================================

def face_to_body_bbox(face_bbox, img_shape, width_scale=3.6, height_top_scale=0.5, height_bottom_scale=5.5):
    """ขยาย face bbox ให้ครอบทั้งตัว"""
    x1, y1, x2, y2 = face_bbox.astype(float)
    face_center_x = (x1 + x2) / 2.0
    face_center_y = (y1 + y2) / 2.0
    face_width = x2 - x1
    face_height = y2 - y1
    
    img_h, img_w = img_shape
    
    half_body_width = (face_width * width_scale) / 2.0
    body_x1 = face_center_x - half_body_width
    body_x2 = face_center_x + half_body_width
    
    body_y1 = face_center_y - (face_height * height_top_scale)
    body_y2 = face_center_y + (face_height * height_bottom_scale)
    
    body_x1 = max(0, body_x1)
    body_y1 = max(0, body_y1)
    body_x2 = min(img_w - 1, body_x2)
    body_y2 = min(img_h - 1, body_y2)
    
    return np.array([body_x1, body_y1, body_x2, body_y2], dtype=np.int32)

def create_overlay(image_rgb, mask, color=[0, 255, 128], alpha=0.5):
    """สร้างภาพ overlay โดยใส่สีทับ mask"""
    overlay = image_rgb.copy()
    mask_bool = mask.astype(bool)
    color_layer = np.zeros_like(image_rgb)
    color_layer[mask_bool] = color
    overlay = cv2.addWeighted(overlay, 1.0, color_layer, alpha, 0)
    return overlay

def create_cutout(image_rgb, mask):
    """สร้างภาพ cutout พื้นหลังโปร่งใส"""
    mask_bool = mask.astype(bool)
    alpha_channel = (mask_bool * 255).astype(np.uint8)
    cutout_rgba = np.dstack((image_rgb, alpha_channel))
    return cutout_rgba

def segment_member(image_bgr, member_name, similarity_threshold=0.45):
    """
    Pipeline หลัก: รับภาพ + ชื่อสมาชิก คืนผลลัพธ์หลายแบบ
    """
    if member_name not in embeddings_db:
        return None, None, None, None, f"❌ ไม่พบ embedding ของ '{member_name}'"
    
    target_embedding = embeddings_db[member_name]
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    img_h, img_w = image_bgr.shape[:2]
    
    # Step 1: Identify all members
    members = identify_all_members(image_bgr, face_analyzer, embeddings_db, similarity_threshold)
    
    target = None
    for m in members:
        if m["name"] == member_name:
            target = m
            break
    
    if target is None:
        return None, None, None, None, f"❌ ไม่พบ {member_name} ในภาพ"
    
    # Step 2: Expand face bbox to body bbox
    face_bbox = np.array(target["bbox"])
    body_bbox = face_to_body_bbox(face_bbox, (img_h, img_w))
    
    # Step 3: SAM 3 Segmentation
    image_pil = Image.fromarray(image_rgb)
    mask = segment_by_box(image_pil, body_bbox.tolist())
    
    # Step 4: Create outputs
    overlay = create_overlay(image_rgb, mask)
    cutout = create_cutout(image_rgb, mask)
    
    # Create annotated image
    annotated = image_rgb.copy()
    for m in members:
        x1, y1, x2, y2 = m["bbox"]
        is_target = (m["name"] == member_name)
        color_box = (0, 255, 0) if is_target else (200, 200, 200)
        thickness = 3 if is_target else 1
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color_box, thickness)
        if m["similarity"]:
            label = f"{m['name']} ({m['similarity']:.2f})"
            cv2.putText(annotated, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_box, 2)
    
    status = f"✅ Found {member_name} (sim={target['similarity']:.3f})"
    
    return overlay, cutout, annotated, mask, status

print("✅ Integration Pipeline ready!")

---
<a id='section-6'></a>
## Section 6: Gradio UI 🎨

สร้าง Web Interface สำหรับใช้งานระบบ

In [ ]:
# =============================================================================
# SECTION 6: Gradio UI
# =============================================================================

IVE_MEMBERS = ["Wonyoung", "Yujin", "Gaeul", "Liz", "Leeseo", "Rei"]

def gradio_segment(input_image, member_name):
    """Gradio callback for image segmentation"""
    if input_image is None:
        return None, None, None, "กรุณาอัพโหลดภาพ"
    
    image_bgr = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    
    overlay, cutout, annotated, mask, msg = segment_member(image_bgr, member_name)
    
    if overlay is None:
        return None, None, None, msg
    
    return annotated, overlay, cutout, msg

def gradio_identify_all(input_image):
    """แสดงทุกคนที่ identify ได้"""
    if input_image is None:
        return None, "กรุณาอัพโหลดภาพ"
    
    image_bgr = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    members = identify_all_members(image_bgr, face_analyzer, embeddings_db)
    
    annotated = input_image.copy()
    info_lines = []
    
    for m in members:
        x1, y1, x2, y2 = m["bbox"]
        cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
        if m["similarity"]:
            label = f"{m['name']} {m['similarity']:.2f}"
            info_lines.append(f"{m['name']}: similarity={m['similarity']:.3f}")
        else:
            label = m["name"]
        cv2.putText(annotated, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    return annotated, "\n".join(info_lines)

# สร้าง Gradio Interface
with gr.Blocks(title="IVE Segmentation") as demo:
    gr.Markdown("# 🎯 IVE Member Segmentation with SAM 3")
    gr.Markdown("อัพโหลดภาพกลุ่ม IVE → เลือกสมาชิก → ระบบ segment ออกมาให้")
    
    with gr.Tabs():
        with gr.TabItem("🔍 Segment Member"):
            with gr.Row():
                with gr.Column(scale=1):
                    img_input = gr.Image(label="อัพโหลดภาพ", type="numpy")
                    member_dropdown = gr.Dropdown(
                        choices=IVE_MEMBERS,
                        value="Wonyoung",
                        label="เลือกสมาชิก"
                    )
                    btn_segment = gr.Button("🔍 Segment", variant="primary")
                    status_text = gr.Textbox(label="Status", interactive=False)
                
                with gr.Column(scale=2):
                    with gr.Row():
                        out_identified = gr.Image(label="Identified")
                        out_overlay = gr.Image(label="Segmented")
                        out_cutout = gr.Image(label="Cutout")
            
            btn_segment.click(
                fn=gradio_segment,
                inputs=[img_input, member_dropdown],
                outputs=[out_identified, out_overlay, out_cutout, status_text]
            )
        
        with gr.TabItem("👥 Identify All"):
            with gr.Row():
                img_input_all = gr.Image(label="อัพโหลดภาพ", type="numpy")
                btn_identify = gr.Button("👥 Identify All", variant="primary")
            with gr.Row():
                out_all = gr.Image(label="All Members")
                out_info = gr.Textbox(label="Info", lines=8)
            
            btn_identify.click(
                fn=gradio_identify_all,
                inputs=[img_input_all],
                outputs=[out_all, out_info]
            )

print("✅ Gradio UI created!")
print("รันด้วย: demo.launch(share=True)")

In [ ]:
# รัน Gradio UI
demo.launch(share=True)

---
<a id='section-7'></a>
## Section 7: Video Inference 🎬

ประมวลผลวิดีโอแบบ Frame-by-Frame พร้อม Tracking

In [ ]:
# =============================================================================
# SECTION 7: Video Inference
# =============================================================================

from collections import deque
import time

class SimpleTracker:
    """Simple IoU-based tracker"""
    
    def __init__(self, iou_threshold=0.5, max_disappeared=5):
        self.iou_threshold = iou_threshold
        self.max_disappeared = max_disappeared
        self.objects = {}
        self.disappeared = {}
        self.next_object_id = 0
    
    def compute_iou(self, box1, box2):
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        xi1 = max(x1_1, x1_2)
        yi1 = max(y1_1, y1_2)
        xi2 = min(x2_1, x2_2)
        yi2 = min(y2_1, y2_2)
        if xi2 <= xi1 or yi2 <= yi1:
            return 0.0
        inter_area = (xi2 - xi1) * (yi2 - yi1)
        box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
        box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = box1_area + box2_area - inter_area
        return inter_area / union_area if union_area > 0 else 0.0
    
    def update(self, detections):
        if len(detections) == 0:
            for obj_id in list(self.disappeared.keys()):
                self.disappeared[obj_id] += 1
                if self.disappeared[obj_id] > self.max_disappeared:
                    del self.objects[obj_id]
                    del self.disappeared[obj_id]
            return self.objects
        
        if len(self.objects) == 0:
            for det in detections:
                self.objects[self.next_object_id] = det
                self.disappeared[self.next_object_id] = 0
                self.next_object_id += 1
        else:
            object_ids = list(self.objects.keys())
            object_bboxes = [self.objects[oid]['bbox'] for oid in object_ids]
            detection_bboxes = [d['bbox'] for d in detections]
            
            iou_matrix = np.zeros((len(object_ids), len(detections)))
            for i, obj_box in enumerate(object_bboxes):
                for j, det_box in enumerate(detection_bboxes):
                    iou_matrix[i, j] = self.compute_iou(obj_box, det_box)
            
            matched_object_ids = set()
            matched_detection_indices = set()
            
            while True:
                max_iou = np.max(iou_matrix)
                if max_iou < self.iou_threshold:
                    break
                max_idx = np.unravel_index(np.argmax(iou_matrix), iou_matrix.shape)
                obj_idx, det_idx = max_idx
                object_id = object_ids[obj_idx]
                self.objects[object_id] = detections[det_idx]
                self.disappeared[object_id] = 0
                matched_object_ids.add(object_id)
                matched_detection_indices.add(det_idx)
                iou_matrix[obj_idx, :] = -1
                iou_matrix[:, det_idx] = -1
            
            for object_id in object_ids:
                if object_id not in matched_object_ids:
                    self.disappeared[object_id] += 1
                    if self.disappeared[object_id] > self.max_disappeared:
                        del self.objects[object_id]
                        del self.disappeared[object_id]
            
            for i, detection in enumerate(detections):
                if i not in matched_detection_indices:
                    self.objects[self.next_object_id] = detection
                    self.disappeared[self.next_object_id] = 0
                    self.next_object_id += 1
        
        return self.objects

class TemporalSmoother:
    """Temporal smoothing for masks"""
    
    def __init__(self, window_size=5):
        self.window_size = window_size
        self.mask_history = deque(maxlen=window_size)
    
    def update(self, mask):
        mask_float = mask.astype(np.float32)
        self.mask_history.append(mask_float)
        if len(self.mask_history) > 0:
            smoothed = np.mean(self.mask_history, axis=0)
            return (smoothed > 0.5).astype(np.uint8)
        return mask
    
    def reset(self):
        self.mask_history.clear()

print("✅ Video processing classes loaded!")

In [ ]:
def process_video(input_path, output_path, target_member, frame_sampling=5):
    """
    ประมวลผลวิดีโอ frame-by-frame
    """
    cap = cv2.VideoCapture(input_path)
    
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {input_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"📹 Video: {width}x{height} @ {fps}fps, {total_frames} frames")
    
    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps / frame_sampling, (width, height))
    
    tracker = SimpleTracker(iou_threshold=0.5, max_disappeared=5)
    smoother = TemporalSmoother(window_size=5)
    
    frame_idx = 0
    processed_count = 0
    
    with tqdm(total=total_frames, desc="Processing") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            if frame_idx % frame_sampling == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Identify members
                members = identify_all_members(frame, face_analyzer, embeddings_db)
                
                # Find target member
                target_detections = [m for m in members if m['name'] == target_member]
                
                # Update tracker
                tracked = tracker.update(target_detections)
                
                # Process each tracked object
                output_frame = frame_rgb.copy()
                
                for obj_id, obj_data in tracked.items():
                    bbox = obj_data['bbox']
                    
                    # Segment
                    image_pil = Image.fromarray(frame_rgb)
                    body_bbox = face_to_body_bbox(np.array(bbox), (height, width))
                    mask = segment_by_box(image_pil, body_bbox.tolist())
                    
                    # Temporal smoothing
                    smoothed_mask = smoother.update(mask)
                    
                    # Visualization
                    color = (255, 105, 180)
                    output_frame = create_overlay(output_frame, smoothed_mask, color, 0.5)
                    
                    # Draw bbox
                    x1, y1, x2, y2 = bbox
                    cv2.rectangle(output_frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(output_frame, target_member, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Write output
                output_bgr = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
                out.write(output_bgr)
                processed_count += 1
            
            frame_idx += 1
            pbar.update(1)
    
    cap.release()
    out.release()
    
    print(f"\n✅ เสร็จสิ้น! ประมวลผล {processed_count} frames")
    print(f"   บันทึกที่: {output_path}")
    
    return output_path

print("✅ Video processing function ready!")

In [ ]:
# ตัวอย่างการใช้งาน (ต้องมีไฟล์วิดีโอ)
# input_video = "path/to/video.mp4"
# output_video = "outputs/segmented_wonyoung.mp4"
# process_video(input_video, output_video, "Wonyoung", frame_sampling=5)

---
## 🎉 สรุป

โปรเจคนี้ประกอบด้วย:
1. ✅ Environment Setup - ติดตั้ง dependencies
2. ✅ Face Embedding Database - สร้าง embeddings สำหรับ IVE
3. ✅ Identity Matching - ระบุตัวตนด้วย Hungarian Algorithm
4. ✅ SAM 3 Engine - Segmentation ด้วย box prompt
5. ✅ Integration Pipeline - เชื่อมโยงทุกอย่างเข้าด้วยกัน
6. ✅ Gradio UI - Web interface สำหรับใช้งาน
7. ✅ Video Inference - ประมวลผลวิดีโอพร้อม tracking

**พร้อมใช้งานบน RTX 6000 (48GB VRAM)!**